In [6]:
import os
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment, EnvironmentSettings

In [ ]:
env = StreamExecutionEnvironment.get_execution_environment()

#settings = EnvironmentSettings.new_instance().in_streaming_mode().use_blink_planner().build()

# create table environment
tbl_env = StreamTableEnvironment.create(stream_execution_environment=env)

# add kafka connector dependency
kafka_jar = "<absolute_path>/flink-sql-connector-kafka-3.3.0-1.20.jar"

tbl_env.get_config()\
        .get_configuration()\
        .set_string("pipeline.jars", "file://{}".format(kafka_jar))

In [ ]:
#######################################################################
# Create Kafka Source Table with DDL
#######################################################################
src_ddl = """
    CREATE TABLE sales_usd (
        `index` STRING,
        `day` STRING,
        `open` DOUBLE,
        `high` DOUBLE,
        `low` DOUBLE,
        `close` DOUBLE,
        `time` AS PROCTIME(),
        `date` AS TO_TIMESTAMP(`day`), 
        WATERMARK FOR `date` AS `date` - INTERVAL '5' SECOND
    ) WITH (
        'connector' = 'kafka',
        'topic' = 'trade_events',
        'properties.bootstrap.servers' = 'localhost:9092',
        'properties.group.id' = 'trade_events_group',
        'format' = 'json'
    );
"""

tbl_env.execute_sql(src_ddl)

# create and initiate loading of source Table
tbl = tbl_env.from_path('sales_usd')

print('\nSource Schema')
tbl.print_schema()


Source Schema
(
  `index` STRING,
  `day` STRING,
  `open` DOUBLE,
  `high` DOUBLE,
  `low` DOUBLE,
  `close` DOUBLE,
  `time` TIMESTAMP_LTZ(3) NOT NULL *PROCTIME* AS PROCTIME(),
  `date` TIMESTAMP(3) *ROWTIME* AS TO_TIMESTAMP(`day`),
  WATERMARK FOR `date`: TIMESTAMP(3) AS `date` - INTERVAL '5' SECOND
)


In [ ]:
#####################################################################
# Define Tumbling Window Aggregate Calculation (Seller Sales Per Minute)
#####################################################################
sql = """
        SELECT
          TUMBLE_START(`time`, INTERVAL '10' SECONDS) as window_start,
          TUMBLE_END(`time`, INTERVAL '10' SECONDS) AS window_end,
          SUM(`close`) AS window_sales
        FROM sales_usd
        GROUP BY
          TUMBLE(`time`, INTERVAL '10' SECONDS)
    """
revenue_tbl = tbl_env.sql_query(sql)

print('\nProcess Sink Schema')
revenue_tbl.print_schema()


Process Sink Schema
(
  `window_start` TIMESTAMP(3) NOT NULL,
  `window_end` TIMESTAMP(3) NOT NULL,
  `window_sales` DOUBLE
)


In [ ]:
###############################################################
# Create Kafka Sink Table
###############################################################
sink_ddl1 = """
    CREATE TABLE sales_euros (
        window_start TIMESTAMP(0),
        window_end TIMESTAMP(0),
        window_sales DOUBLE
    ) WITH (
        'connector' = 'kafka',
        'topic' = 'top_k_trades',
        'properties.bootstrap.servers' = 'localhost:9092',
        'format' = 'json'
    )
"""
r = tbl_env.execute_sql(sink_ddl1)

In [13]:
# write time windowed aggregations to sink table
revenue_tbl.execute_insert('sales_euros').wait()
tbl_env.execute('windowed-sales')

Py4JJavaError: An error occurred while calling o148.executeInsert.
: java.lang.NullPointerException: metadataHandlerProvider
	at java.base/java.util.Objects.requireNonNull(Objects.java:259)
	at org.apache.calcite.rel.metadata.RelMetadataQueryBase.getMetadataHandlerProvider(RelMetadataQueryBase.java:122)
	at org.apache.calcite.rel.metadata.RelMetadataQueryBase.revise(RelMetadataQueryBase.java:118)
	at org.apache.calcite.rel.metadata.RelMetadataQuery.getPulledUpPredicates(RelMetadataQuery.java:844)
	at org.apache.calcite.rel.rules.ReduceExpressionsRule$ProjectReduceExpressionsRule.onMatch(ReduceExpressionsRule.java:307)
	at org.apache.calcite.plan.AbstractRelOptPlanner.fireRule(AbstractRelOptPlanner.java:337)
	at org.apache.calcite.plan.hep.HepPlanner.applyRule(HepPlanner.java:556)
	at org.apache.calcite.plan.hep.HepPlanner.applyRules(HepPlanner.java:420)
	at org.apache.calcite.plan.hep.HepPlanner.executeRuleInstance(HepPlanner.java:243)
	at org.apache.calcite.plan.hep.HepInstruction$Rul